In [25]:
#importing jupyterDash
from jupyter_plotly_dash import JupyterDash

#importing dash libraries
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State

#importing python libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import base64

#Importing custom CRUD module for AAC database
from AnimalShelterClass import AnimalShelter



###########################
# Data Manipulation / Model
###########################

#setting up our user and AnimalShelter CRUD object
username = 'aacuser'
password = 'myPassword'
shelter = AnimalShelter(username, password)

#using pandas to read the objects from the cursor into records
df = pd.DataFrame.from_records(shelter.read({}))

#Logo information
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())




#########################
# Dashboard Layout / View
#########################
app = JupyterDash('DataTable')

app.layout = html.Div([
    #Logo with hyperlink
    html.A(
        href='https://www.snhu.edu',
        children=[
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height=100, width=100),
        ]
    ),
    
    #Title
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    #chart filter dropdown formatting
    dcc.Dropdown(
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Wilderness Rescue', 'value': 'Wilderness Rescue'},
                {'label': 'Individual Tracking', 'value': 'Individual Tracking'},
                {'label': 'No Filter', 'value': 'No Filter'},
            ],
            value='No Filter',
            id='filter-choice'
        ),
    
    #the main data table
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,

    ),
    html.Br(),
    html.Hr(),
    
    #DIV that places our graph and geolocation side by side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 
        #Graph placeholder
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
                 
        #Map placeholder
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Div('Creator: Jared Smith')
    
])




#########################
# Interactivity implementation (callback functions)
#########################


# Callback for cat/dog filtering, each button acts like a boolean turning cat/dog filtering on or off
@app.callback(
            Output('datatable-interactivity', "data"),
            [Input('filter-choice', 'value')])
def on_click(value):
    # whenever both buttons have been pressed and even or odd number of times
    # generates an unnfiltered list
    if (value=='No Filter'):
        df = pd.DataFrame(list(shelter.read({})))
    
    #if button1 (for cats) has been clicked an odd number of times, filters to cats
    elif (value=='Water Rescue'):
        df = pd.DataFrame(list(shelter.read({"breed": {"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks": {"$gte": 26}, "age_upon_outcome_in_weeks": {"$lte": 156}})))
    
    #if button2 (for dogs) has been clicked an odd number of times, filters to dogs
    elif (value=='Wilderness Rescue'):
        df = pd.DataFrame(list(shelter.read({"breed": {"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 26}, "age_upon_outcome_in_weeks": {"$lte": 156}})))
    
    elif (value=='Individual Tracking'):
        df = pd.DataFrame(list(shelter.read({"breed": {"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": {"$gte": 20}, "age_upon_outcome_in_weeks": {"$lte": 300}})))
        
    return df.to_dict('records')

# Callback function for selecting a cell
@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Callback function for pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(data_frame=dff, names='breed')
    return [
        dcc.Graph(figure=fig)
    ]


# Callback function for the geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app